In [2]:
from pathlib import Path
from pl_models.DEA import DAE_LitModel
import matplotlib.pyplot as plt
import nibabel as nib
import torch
from torchvision import transforms
from BIDS import NII
from BIDS.core.np_utils import np_dice
import numpy as np
from BIDS.core.np_utils import np_map_labels, Label_Map
from sklearn.manifold import TSNE
from dataloader.datasets.dataset_csv import model_map_to_segmentation_map, segmentation_map_to_model_map 
#3D
def load_nifti(nii):
        
        from_im = nii.get_seg_array()
        #range_print(torch.from_numpy(from_im.astype(float)),41,41)
        #print(from_im[143,95,:])
        print(np.unique(from_im))
        from_im = np_map_labels(arr=from_im,label_map={50: 49})
        from_im = from_im.astype(float)
        from_im = np_map_labels(arr=from_im,label_map={49: 1})
            
        from_im = (from_im-0.5)/0.5
        from_im = torch.from_numpy(from_im).unsqueeze(0).unsqueeze(0)
        print("from_im shape:", from_im.shape)
        from_im = from_im.to(torch.float32)
        
        print("tesnor", from_im.shape)
        return from_im # , "index": target, "cls_labels": target}
# def map_to_binary(y):
#         #y = y.astype(float)
#         labelmap = {i: 1 for i in range(41, 50)} #  labelmap = {i: round((i - 40)/9, ndigits=2) for i in range(41, 50)}
#         #print(labelmap)
#         return np_map_labels(y, labelmap)
# def segmentation_map_to_model_map(y):
#         y = y.astype(float)
#         labelmap = {i: round((i - 40)/9, ndigits=5) for i in range(41, 50)} #  labelmap = {i: round((i - 40)/9, ndigits=2) for i in range(41, 50)}
#         print(labelmap)
#         return np_map_labels(y, labelmap)
# def model_map_to_segmentation_map(x):
#     x =(x+1)/2
#     x *= 9
#     x = np.round(x+40)# round nearest int
#    # x = x+40
#     x = np_map_labels(x, {40: 0})
#     return x

# def model_map_to_binary_map(x):
#     x = np.round(x)
#     return x

#"/media/DATA/martina_ma/cutout/verse040/verse040_023_subreg_cropped.nii.gz",#
#/media/DATA/martina_ma/cutout/tri177/tri177_007_subreg_cropped.nii.gz 
#in training set: /media/DATA/martina_ma/cutout/verse040/verse040_018_subreg_cropped.nii.gz
#not in train and not in val: /media/DATA/martina_ma/cutout/verse040/verse040_023_subreg_cropped.nii.gz
# Test sample /media/DATA/martina_ma/cutout/tri050/tri050_021_subreg_cropped.nii.gz
# Train sample in old seg /media/DATA/martina_ma/cutout/verse529/verse529_014_new_subreg_cropped.nii.gz
nii = NII.load(
    "/media/DATA/martina_ma/cutout_corpus/ctfu00699/ctfu00699_17_subreg_corpus.nii.gz",
    True,
)
from_im = load_nifti(nii)

checkpoint_path ="/media/DATA/martina_ma/dae/lightning_logs/DAE_3D_corpus/version_20/checkpoints/epoch=25-step=197990d_score=1.0000_d_score_latest.ckpt"#"/media/DATA/martina_ma/dae/lightning_logs/DAE_3D_corpus/version_17/checkpoints/epoch=0-step=7615d_score=0.7215_d_score_latest.ckpt"#"/media/DATA/martina_ma/dae/lightning_logs/DAE_3D_cleaned/version_5/checkpoints/epoch=11-step=113760d_score=0.9303_d_score_latest.ckpt"#"/media/DATA/martina_ma/dae/lightning_logs/DAE_3D_95_old_verse_w_norm/version_11/checkpoints/epoch=0-step=1488_d_score_latest.ckpt"#"/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_232/checkpoints/epoch=283-step=27190_latest.ckpt"# "/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_133/checkpoints/epoch=3-step=310_latest.ckpt"
#"/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_135/checkpoints/epoch=27-step=2775_latest.ckpt"#/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_133/checkpoints/epoch=3-step=310_latest.ckpt"#"/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_118/checkpoints/epoch=80-step=243_latest.ckpt"#"/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_116/checkpoints/epoch=81-step=246_latest.ckpt"#"/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_108/checkpoints/epoch=47-step=144_latest.ckpt"#"/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_105/checkpoints/epoch=69-step=210_latest.ckpt"#"/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_63/checkpoints/epoch=79-step=240_latest.ckpt"#"/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_60/checkpoints/epoch=71-step=216_latest.ckpt"#lightning_logs/DAE_NAKO_256/version_26/checkpoints/epoch=404-step=405_latest.ckpt"
device = "cuda:0"  # or "cpu" if you want to use CPU
assert Path(checkpoint_path).exists()
model = DAE_LitModel.load_from_checkpoint(checkpoint_path)
model.ema_model.eval()
model.ema_model.to(device)

img_data = from_im.to(device)
print(np.unique(img_data[0][0].cpu().numpy()))

print("torch:",img_data.shape)
# Ensure that the input has a single channel (if the model expects 1 channel)
if img_data.shape[0] != 1:
    raise Exception()
    #img_data = img_data.mean(1, keepdim=True)

print("mean:",img_data.shape)


cond = model.encode(img_data)
print("cond")
print("cond",cond.shape)

# Decode the encoded representation to obtain the reconstructed 2D image
T = 200  # Adjust T as needed
stoch = model.encode_stochastic(img_data.to(device), cond, T=T)
print("stoch shape:",stoch.shape)
xT = model.render(stoch, cond, T=20) #pred = model.render(xT, cond, T=20)
reconstructed_img = xT#[0]#(xT[0] + 1) / 2  # De-normalize if needed
print("shape:",reconstructed_img.shape)
# Move the reconstructed_img to CPU
# Set values smaller than 0.1111 to 0, and others to 1
reconstructed_img = reconstructed_img.cpu()
#reconstructed_img = torch.where(reconstructed_img < 0.11111, torch.tensor(0.0), torch.tensor(1.0))
print(np.unique(reconstructed_img))

#reconstructed_img = torch.clamp(reconstructed_img, min=0)
#print(np.unique(img_slice))

r = reconstructed_img[0][0].numpy().copy()
reconstructed_img = (r+1)/2
print(np.unique(reconstructed_img))
reconstructed_img = np.round(reconstructed_img)
print(np.unique(reconstructed_img))

r = reconstructed_img.copy()

i = img_data[0][0].cpu().numpy().copy()
i = model_map_to_segmentation_map(i)
print(r.shape)
#r = r*9
#r = np.round(r)
re = r#+ 40
#print(np.unique(re))
#re = np_map_labels(re, {40: 0})
print(np.unique(re))
nii.set_array_(re).save("Reconstructions/Reconstructed_ctfu00699_017_corpus_float_T200_T20.nii.gz")
print(np.unique(nii.get_array()))
re = np_map_labels(re, {1: 49})
print(np_dice(i, re, binary_compare= True))
print(np.unique(i))
#print(np.unique(reconstructed_img[0][0].numpy()))
print(np.unique(re))


[ 0 49]
from_im shape: torch.Size([1, 1, 128, 96, 128])
tesnor torch.Size([1, 1, 128, 96, 128])
Model params: 1.27 M
[-1.  1.]
torch: torch.Size([1, 1, 128, 96, 128])
mean: torch.Size([1, 1, 128, 96, 128])
cond
cond torch.Size([1, 512])
stoch shape: torch.Size([1, 1, 128, 96, 128])
shape: torch.Size([1, 1, 128, 96, 128])
[-1.         -0.99999994 -0.9999998  ...  0.9999988   0.9999998
  1.        ]
[0.0000000e+00 2.9802322e-08 8.9406967e-08 ... 9.9999940e-01 9.9999988e-01
 1.0000000e+00]
[0. 1.]
(128, 96, 128)
[0. 1.]
[ ] 'set_array' with different dtype: from uint8 to float32
[*] Save Reconstructions/Reconstructed_ctfu00699_017_corpus_float_T200_T20.nii.gz as uint8
[0 1]
0.9964997499821416
[ 0. 49.]
[ 0. 49.]
